#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://docs.smith.langchain.com/pricing/faq")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/pricing/faq', 'title': 'Frequently Asked Questions | 🦜️🛠️ LangSmith', 'description': 'Questions and Answers', 'language': 'en'}, page_content="\n\n\n\n\nFrequently Asked Questions | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingPlansFAQReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referencePricingFAQOn this pageFrequently Asked Questions\nQuestions and Answers\u200b\nI’ve been using LangSmith since before pricing took effect for new users. When will pricing go into effect for my account?\u200b\nIf you’ve been using LangSmith already, your usage will be billable starting in

In [16]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
documents = text_splitter.split_documents(docs)

In [23]:
type(documents[0])

langchain_core.documents.base.Document

In [11]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model = 'mxbai-embed-large')

In [12]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [13]:
vectorstoredb

In [15]:
## Query From a vector db
query="ingested event"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'What is a trace?\u200b\nA trace is one complete invocation of your application chain or agent, evaluator run, or playground run. Here is an example of a single trace.\nWhat is an ingested event?\u200b\nAn ingested event is any distinct, trace-related data sent to LangSmith. This includes:'

In [18]:
print(len(result))

4


In [25]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model = 'gemma3:1b')
llm.invoke('Hi How are you?')

"Hi there! I'm doing well, thanks for asking. As an AI, I don't experience feelings in the same way humans do, but my systems are running smoothly and I'm ready to help. 😊 \n\nHow about you? How’s your day going so far?"

In [22]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| OllamaLLM(model='gemma3:1b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [24]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"I’ve hit my rate or usage limits. What can I do?",
    "context":[Document(page_content="When you first sign up for a LangSmith account, you get a Personal organization that is limited to 5000 monthly traces. To continue sending traces after reaching this limit, upgrade to the Developer or Plus plans by adding a credit card. Head to Plans and Billing to upgrade.")]
})

'To continue sending traces after reaching the 5000 monthly trace limit, you need to upgrade to the Developer or Plus plans.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [26]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [27]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [28]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002D9F908B2E0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [29]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"I’ve hit my rate or usage limits. What can I do?"})
response

{'input': 'I’ve hit my rate or usage limits. What can I do?',
 'context': [Document(id='0c974f4c-f8c3-4d40-87a0-ccb73d9c91eb', metadata={'source': 'https://docs.smith.langchain.com/pricing/faq', 'title': 'Frequently Asked Questions | 🦜️🛠️ LangSmith', 'description': 'Questions and Answers', 'language': 'en'}, page_content='I’ve hit my rate or usage limits. What can I do?\u200b\nWhen you first sign up for a LangSmith account, you get a Personal organization that is limited to 5000 monthly traces. To continue sending traces after reaching this limit, upgrade to the Developer or Plus plans by adding a credit card. Head to Plans and Billing to upgrade.\nSimialrly, if you’ve hit the rate limits on your currnt plan, you can upgrade to a higher plan to get higher limits, or reach out to support@langchain.dev with questions.'),
  Document(id='671b4f86-6e46-4596-8ca2-2407509060d1', metadata={'source': 'https://docs.smith.langchain.com/pricing/faq', 'title': 'Frequently Asked Questions | 🦜️🛠️ Lan

In [32]:
print(response['answer'])

Here’s a breakdown of the information from the context regarding your usage limits:

*   **Developer Account:** You can upgrade to the Plus or Enterprise plan.
*   **Billing:** You can set usage limits if you choose to limit the maximum charges you could incur in any given month.
*   **Tracking Usage:** You’ll be billed on the first of the month for traces submitted in the previous month.
*   **Spending Limits:** Yes, you can set limits on how much you spend on tracing.
*   **Data Storage:** Your data is stored.
*   **Security Frameworks:** LangSmith complies with various security frameworks.
*   **Training:** Yes, LangSmith can train on the data you send.
*   **Community Forum:** Available on the CommunityLangChain Forum.
*   **Twitter:** Available on Twitter.
*   **GitHub:** Available on GitHub.
*   **CodeLangSmith:** Available on CodeLangSmith.


In [33]:
response['context']

[Document(id='0c974f4c-f8c3-4d40-87a0-ccb73d9c91eb', metadata={'source': 'https://docs.smith.langchain.com/pricing/faq', 'title': 'Frequently Asked Questions | 🦜️🛠️ LangSmith', 'description': 'Questions and Answers', 'language': 'en'}, page_content='I’ve hit my rate or usage limits. What can I do?\u200b\nWhen you first sign up for a LangSmith account, you get a Personal organization that is limited to 5000 monthly traces. To continue sending traces after reaching this limit, upgrade to the Developer or Plus plans by adding a credit card. Head to Plans and Billing to upgrade.\nSimialrly, if you’ve hit the rate limits on your currnt plan, you can upgrade to a higher plan to get higher limits, or reach out to support@langchain.dev with questions.'),
 Document(id='671b4f86-6e46-4596-8ca2-2407509060d1', metadata={'source': 'https://docs.smith.langchain.com/pricing/faq', 'title': 'Frequently Asked Questions | 🦜️🛠️ LangSmith', 'description': 'Questions and Answers', 'language': 'en'}, page_co